In [30]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

torch.manual_seed(1234)

# Data Preprocessing to include label and shift it one place up

In [23]:
#Preprocessing
nepse_data = pd.read_csv("nepse_data_for_training.csv", index_col=0)
nepse_data.set_index("date", inplace=True)

In [24]:
nepse_data.head(10)

,nepse_index,momentum,EMA,MACD,RSI,STOCHRSI,ROI,ATR,ADX,williams,CCI
date,,,,,,,,,,,
2018-07-15,1814.430054,-84.819946,1837.585960,-6.762209,0.275102,0.000000,1.014784,125.479980,-5.770594,0.675964,1829.150024
2018-07-16,1846.719971,-51.119995,1839.013105,-5.540793,0.581367,0.063240,0.999918,124.069946,-5.486751,0.412026,1839.443319
2018-07-17,1793.609985,-17.810059,1831.936064,-8.254247,0.411728,0.214111,1.029163,72.949951,-5.208680,0.728033,1804.699992
2018-07-18,1793.609985,19.839966,1825.974203,-10.271957,0.803768,1.000000,1.027637,72.949951,-4.936359,0.728033,1804.699992
2018-07-19,1808.219971,-6.210083,1823.217136,-10.680821,1.648653,1.000000,1.015388,53.109985,-4.669757,0.724911,1816.183309
2018-07-22,1773.650024,-73.069946,1815.530928,-13.367648,0.534899,0.099578,1.026838,73.069946,-0.104037,1.000000,1798.006673
2018-07-23,1773.650024,-19.959961,1809.044512,-15.310002,0.166629,0.000000,1.019757,34.569946,0.053211,1.000000,1785.173340
2018-07-24,1823.780029,30.170044,1811.324363,-12.983284,1.872725,1.000000,0.987666,50.130005,2.625605,0.000000,1807.070028
2018-07-25,1807.260010,-0.959961,1810.696082,-12.231167,1.267176,0.645068,0.996464,50.130005,2.719903,0.329544,1801.563354


In [226]:
nepse_data['Label'] = nepse_data['nepse_index'].rolling(window=2).apply(lambda x: 1 if x[1]>x[0] else 0)
nepse_data.Label = nepse_data.Label.shift(-1)
nepse_data.Label[-1] = 0
nepse_data.head(10)

,nepse_index,momentum,EMA,MACD,RSI,STOCHRSI,ROI,ATR,ADX,williams,CCI,Label
date,,,,,,,,,,,,
2018-07-15,1814.430054,-84.819946,1837.585960,-6.762209,0.275102,0.000000,1.014784,125.479980,-5.770594,0.675964,1829.150024,1.0
2018-07-16,1846.719971,-51.119995,1839.013105,-5.540793,0.581367,0.063240,0.999918,124.069946,-5.486751,0.412026,1839.443319,0.0
2018-07-17,1793.609985,-17.810059,1831.936064,-8.254247,0.411728,0.214111,1.029163,72.949951,-5.208680,0.728033,1804.699992,0.0
2018-07-18,1793.609985,19.839966,1825.974203,-10.271957,0.803768,1.000000,1.027637,72.949951,-4.936359,0.728033,1804.699992,1.0
2018-07-19,1808.219971,-6.210083,1823.217136,-10.680821,1.648653,1.000000,1.015388,53.109985,-4.669757,0.724911,1816.183309,0.0
2018-07-22,1773.650024,-73.069946,1815.530928,-13.367648,0.534899,0.099578,1.026838,73.069946,-0.104037,1.000000,1798.006673,0.0
2018-07-23,1773.650024,-19.959961,1809.044512,-15.310002,0.166629,0.000000,1.019757,34.569946,0.053211,1.000000,1785.173340,1.0
2018-07-24,1823.780029,30.170044,1811.324363,-12.983284,1.872725,1.000000,0.987666,50.130005,2.625605,0.000000,1807.070028,0.0
2018-07-25,1807.260010,-0.959961,1810.696082,-12.231167,1.267176,0.645068,0.996464,50.130005,2.719903,0.329544,1801.563354,1.0


# From here on, data is prepared to be fed to pytorch network

The dataset is prepared by inheriting Dataset class, and DataLoader class is used for batching. 


In [67]:
#DataLoader class
class NepseDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.features = dataset.loc[:, 'momentum':'CCI'].as_matrix()
        self.label = dataset['Label'].as_matrix()
        self.transform = transform
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        tech_indicators = self.features[idx,:]
        label = self.label[idx]
        sample = {'features': tech_indicators, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample
    
        
        

In [287]:
class ToTensor(object):

    def __call__(self, sample):
        features, label = sample['features'], sample['label']

        return {'features': torch.from_numpy(features).float(),
                'label': int(label)}

# Our Neural Network
 The network consists of CNN which extracts features from news, the output of which is concatenated with the technical indicators and fed to lstm. The output is then fed to softmax to predict the rise or fall in nepse for the following day.

In [333]:

class StockNN(nn.Module):
    def __init__(self, filter_sizes, drop_prob, embedding_dim):
        super(StockNN, self).__init__()
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = 2, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        
        
    def forward(self,x,h):
        
        #x_cnn = x[-1]
        #x_cnn = x_cnn.reshape(1,1,x_cnn.shape[0], x_cnn.shape[1])
        #conved = [F.relu(conv(x_cnn)).squeeze(3) for conv in self.convs]
        #pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #cat = self.dropout(torch.cat(pooled, dim = 1).squeeze())
        #cat = torch.cat([x[0],cat])
        
        lstm = nn.LSTM(x.shape[1], 256, 2, dropout=self.drop_prob, batch_first=True)
        out, (h,c) = lstm(x.view(x.shape[0],1,-1),h)
        linear = nn.Linear(out.shape[2], 2)
        hidden_2_risefall = linear(out.view(out.shape[0],-1))
        rise_fall = F.softmax(hidden_2_risefall, dim=1)
        
        return rise_fall
    
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(2, n_seqs, 256).zero_(),
                weight.new(2, n_seqs, 256).zero_())


# Function  to prepare Training and validation set

In [334]:
def train_validate_set(data,val_frac=0.2):
    m = data.shape[0]
    val_idx = int(m*(1-val_frac))
    test_data, val_data = NepseDataset(data.head(val_idx),ToTensor()), NepseDataset(data.tail(m - val_idx),ToTensor())
    return test_data, val_data

td, vd = train_validate_set(nepse_data)
print(len(td), len(vd))
    

176 45


# Training the StockNN


In [335]:
#Have to check this one
def train(NN, data, val_frac=0.2, max_epochs=10, batch_size=10, learning_rate=0.01):
    NN.train()
    opt = torch.optim.Adam(NN.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    train_data, validate_data = train_validate_set(data,val_frac)
    dataloader = DataLoader(train_data, batch_size=batch_size)
    for e in range(max_epochs):
        h = test_net.init_hidden(batch_size)
        
        for i_batch, batched_sample in enumerate(dataloader):
            h = tuple([each.data for each in h])
            NN.zero_grad()
            out = test_net(batched_sample['features'],h)
            loss = criterion(out,batched_sample['label'])
            loss.backward()
            
            opt.step()
            print(loss.item())


  

In [336]:
stock_net = StockNN([2,3,4], 0.1, 100)

In [337]:
print(type(nepse_dataset))
train(stock_net, nepse_data, max_epochs= 20, batch_size=8)

<class '__main__.NepseDataset'>
0.7021315097808838
0.694952130317688
0.6919906139373779
0.7007578611373901
0.6925362944602966
0.6935933232307434
0.6989055275917053
0.6957736611366272
0.6957530975341797
0.69268798828125
0.6991579532623291
0.687499463558197
0.6922959685325623
0.698452353477478
0.6996440887451172
0.6950286030769348
0.6925271153450012
0.6971721053123474
0.6942476630210876
nan
nan
0.694265604019165
0.6912586688995361
0.6883021593093872
0.6942590475082397
0.696923017501831
0.6948468089103699
0.6918845772743225
0.691628098487854
0.6947293877601624
0.6893396377563477
0.6925091743469238
0.6953598260879517
0.6970847845077515
0.6953074336051941
0.697055459022522
0.6919090151786804
0.6938043832778931
0.6960610747337341
0.6859021186828613
0.6938108801841736
nan
nan
0.692953884601593
0.7024163603782654
0.686604380607605
0.6943836212158203
0.6705788373947144
0.6947464942932129
0.6980284452438354
0.6896266341209412
0.6879708766937256
0.7023295164108276
0.6955968141555786
0.69997966289